In [ ]:
import cellCnn
import importlib

importlib.reload(cellCnn)
import random
import numpy as np
### (from: https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/NK_cell_ungated.ipynb ) AND https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/PBMC.ipynb
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt

from cellCnn.ms.utils.helpers import get_min_mean_from_clusters, get_chunks
from cellCnn.utils import mkdir_p
from cellCnn.plotting import plot_results
from cellCnn.ms.utils.helpers import get_chunks
from cellCnn.ms.utils.helpers import print_regression_model_stats
from cellCnn.plotting import plot_results
from cellCnn.utils import mkdir_p
from cellCnn.utils import save_results
from cellCnn.ms.utils.helpers import get_fitted_model
from cellCnn.ms.utils.helpers import split_test_valid_train
from cellCnn.ms.utils.helpers import calc_frequencies, get_chunks_from_df

import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [ ]:
#%reset

In [ ]:
##### state vars
cytokines = ['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28', 'CD3',
             'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10',
             'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a']
infile = 'cohort_denoised_clustered_diagnosis_patients.csv'
indir = 'data/input.nosync'
outdir = 'out_ms_default'
rand_seed = 123
train_perc = 0.7
test_perc = 0.3
batch_size_pheno = 840 # deprecated  # so a size of 8425 is about equally sized in batches
batch_size_cd4 = 550  # deprecated #so a size of 550 gets me 16 batches for cd4
## information from ms_data project
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}
outfolder = 'mtl_1'
mkdir_p(outfolder)

In [ ]:
np.random.seed(rand_seed)
mkdir_p(outdir)
df = pd.read_csv(os.path.join(indir, infile), index_col=0)
df = df.drop_duplicates()  ### reduces overfitting at cost of fewer data
df.shape
##### no duplicates in

In [ ]:
# pitch: key = gate_source, value = dict{diagnosis: df OR freq?}
rrms_df = df[df['diagnosis'] == 'RRMS']
rrms_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                    rrms_df.groupby('gate_source')}
nindc_df = df[df['diagnosis'] == 'NINDC']
nindc_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                     nindc_df.groupby('gate_source')}

In [ ]:
#### here we could see freq differences across the 2 groups
print('Frequencies: ')
rrms_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df in
                      rrms_patients2df.items()}
nindc_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df
                       in nindc_patients2df.items()}
print('DONE')
# we got 31 patients each

In [ ]:
patients_clusters_conf_table = pd.crosstab(df['gate_source'], df['cluster'])
sns.heatmap(patients_clusters_conf_table, annot=False, vmax=100)
plt.show()
#plt.savefig('images/patient_vs_cluster_conf_table.png')
patients_clusters_conf_table

In [ ]:
importlib.reload(cellCnn.ms.utils.helpers)
from cellCnn.ms.utils.helpers import *

# batch_sizes = [40,80,120, 160]
batch_size_dict = dict()
batch_sizes = [120]
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_cd8 = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           batch_size=batch_size)
    selection_pool_nindc_cd8 = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           batch_size=batch_size)

    # make sure list are equally long:
    if len(selection_pool_rrms_cd8) > len(selection_pool_nindc_cd8):
        selection_pool_rrms_cd8 = selection_pool_rrms_cd8[:len(selection_pool_nindc_cd8)]
    elif len(selection_pool_rrms_cd8) < len(selection_pool_nindc_cd8):
        selection_pool_nindc_cd8 = selection_pool_nindc_cd8[:len(selection_pool_rrms_cd8)]

    all_chunks = selection_pool_rrms_cd8 + selection_pool_nindc_cd8
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    freqs = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]
    batch_size_dict[batch_size] = (X, freqs, Y)

print('DONE: batches created')

In [ ]:
### CD8 Freq regression task + MTL
model_container_cd8 = []
stats_dict_reg_cd8 = dict()
outdir_cd8 = 'mtl/class/cd8'
mkdir_p(outdir_cd8)

In [57]:
importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
importlib.reload(cellCnn)
importlib.reload(cellCnn.utils)
from cellCnn.utils import *
from cellCnn.ms.utils.helpers import *
from cellCnn import *


kf_split = 2
for batch_size, values, in batch_size_dict.items():
    # for regression task stratified is wrong since there are no classes
    freq_idx =3
    X, cd8, y = values[0], values[1], values[2]
    cd8 = [series[freq_idx] for series in cd8]

    X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid, y_test, y_train, y_valid = split_test_train_valid(
        X, cd8, y,
        train_perc=train_perc,
        test_perc=test_perc,
        valid_perc=0.5)
    outdir_pheno_reg_cd8 = f'{outdir_cd8}/models/ms_class_cd8_{batch_size}'

    # todo nsubset parameter maybe different
    model = get_fitted_model(X_train, X_valid, [y_train, cd8_train], [cd8_valid, y_valid],
                             nsubset=100,
                             nfilters=[3, 15, 25, 35], coeff_l1=0,
                             max_epochs=100, nrun=10, learning_rate=None,
                             ncell=200,
                             per_sample=True, regression=False,
                             outdir=outdir_pheno_reg_cd8, verbose=False, mtl_tasks=2)
    model_container_cd8.append(model)
print('DONE building models')
print('DONE class with per_sampel true')

ERROR! Session/line number was not unique in database. History logging moved to new session 976


KeyboardInterrupt: 

In [25]:
accs = []

# what predict does is: Iterating over the best 3 nets and return an array per net with the predictions for all samples!
bs_idx = 1
for i, model in enumerate(model_container_cd8):
    stats_file = open(f"{outdir_cd8}/stats_class_{model.ncell}_{model.nsubset}.txt", "w+")
    stats_file.write(f"Model {i}; Batchsize: {batch_sizes[bs_idx-1]}\n")
    if bs_idx % kf_split == 0: ## when the kf split amount is reaches the next batchsize is applied
        bs_idx = bs_idx +1
    test_pred = model.predict(X_test)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)
    #print_regression_model_stats(test_pred, b_test)
    test_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in test_pred[0]]
    train_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in train_pred[0]]
    valid_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in valid_pred[0]]
    acc_test = accuracy_score(y_test,  test_pred_abs)
    acc_train = accuracy_score(y_train,  train_pred_abs)
    acc_valid = accuracy_score(y_valid,  valid_pred_abs)
    accs.append(acc_test)

    stats_file.write('Desease: \n')
    stats_file.write(f'Acc y test {acc_test}\n')
    stats_file.write(f'Acc y train {acc_train}\n')
    stats_file.write(f'Acc y valid {acc_valid}\n')

    mse_test_cd8 = mean_squared_error(cd8_test, test_pred[1])
    mse_train_cd8 = mean_squared_error(cd8_train, train_pred[1])
    mse_valid_cd8 = mean_squared_error(cd8_valid, valid_pred[1])
    mses.append(mse_test_cd8)

    stats_file.write('Cd8: \n')
    stats_file.write(f'MSE y test {mse_test_cd8}\n')
    stats_file.write(f'MSE y train {mse_train_cd8}\n')
    stats_file.write(f'MSE y valid {mse_valid_cd8}\n')
    stats_file.write('\n')
    stats_file.close()
print('DONE')

DONE


In [75]:
### CD8 Freq regression task + MTL
model_container_cd8_class = []
stats_dict_cd8_class = dict()

In [76]:
importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
importlib.reload(cellCnn)
importlib.reload(cellCnn.utils)
from cellCnn.utils import *
from cellCnn.ms.utils.helpers import *
from cellCnn import *


kf_split = 2
for batch_size, values, in batch_size_dict.items():
    # for regression task stratified is wrong since there are no classes
    freq_idx =3
    X, cd8, y = values[0], values[1], values[2]
    cd8 = [series[freq_idx] for series in cd8]

    X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid, y_test, y_train, y_valid = split_test_train_valid(
        X, cd8, y,
        train_perc=train_perc,
        test_perc=test_perc,
        valid_perc=0.5)
    outdir_pheno_reg_cd8 = f'{outdir_cd8}/models/ms_class_cd8_{batch_size}_psFalse'

    # todo nsubset parameter maybe different
    # yet you need to put phenotype data in frist place
    model = get_fitted_model(X_train, X_valid, [y_train, cd8_train], [y_valid, cd8_valid],
                             nsubset=100,
                             nfilters=[3, 15, 25, 35], coeff_l1=0,
                             max_epochs=100, nrun=10, learning_rate=None,
                             ncell=200,
                             per_sample=False, regression=False,
                             outdir=outdir_pheno_reg_cd8, verbose=False, mtl_tasks=2)
    model_container_cd8_class.append(model)
print('DONE building models')
print('DONE class with per_sampel false')

1/1 [==============================] - 0s 26ms/step - loss: 0.6987 - output_loss: 0.6944 - output_freq_1_loss: 0.0041 - output_accuracy: 0.4646 - output_mean_squared_error: 0.2506 - output_freq_1_accuracy: 0.0000e+00 - output_freq_1_mean_squared_error: 0.0041
DONE building models
DONE class with per_sampel false


/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [77]:
importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
importlib.reload(cellCnn)
importlib.reload(cellCnn.utils)
from cellCnn.utils import *
from cellCnn.ms.utils.helpers import *
from cellCnn import *


kf_split = 2
for batch_size, values, in batch_size_dict.items():
    # for regression task stratified is wrong since there are no classes
    freq_idx =3
    X, cd8, y = values[0], values[1], values[2]
    cd8 = [series[freq_idx] for series in cd8]

    X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid, y_test, y_train, y_valid = split_test_train_valid(
        X, cd8, y,
        train_perc=train_perc,
        test_perc=test_perc,
        valid_perc=0.5)
    outdir_pheno_reg_cd8 = f'{outdir_cd8}/models/ms_class_cd8_{batch_size}_psFalse'

    # todo nsubset parameter maybe different
    # yet you need to put phenotype data in frist place
    model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                             nsubset=100,
                             nfilters=[3, 15, 25, 35], coeff_l1=0,
                             max_epochs=100, nrun=10, learning_rate=None,
                             ncell=200,
                             per_sample=False, regression=False,
                             outdir=outdir_pheno_reg_cd8, verbose=False, mtl_tasks=1)
    model_container_cd8_2.append(model)
print('DONE building models')
print('DONE class with per_sampel false')



7/7 [==============================] - 0s 5ms/step - loss: 0.6927 - accuracy: 0.4747


NameError: name 'model_container_cd8_2' is not defined